In [1]:
# STEP 1: Required Libraries Install
!pip install langchain langchain-community langchain-openai openai requests


In [2]:
!pip install requests==2.32.4


  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.


In [3]:
# STEP 2: Import Libraries (Safe Imports)
import json
import requests
from typing import List, Dict


In [4]:
# STEP 3: Upload JSON Files to Colab
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# loading data set
BASE_PATH = "/content/drive/MyDrive/Agentic_AI_Travel_Planner"

FLIGHTS_FILE = BASE_PATH + "/content/drive/MyDrive/Agentic_AI_Travel_Planner/flights.json"
HOTELS_FILE = BASE_PATH + "/content/drive/MyDrive/Agentic_AI_Travel_Planner/hotels.json"
PLACES_FILE = BASE_PATH + "/content/drive/MyDrive/Agentic_AI_Travel_Planner/places.json"

print(FLIGHTS_FILE)

/content/drive/MyDrive/Agentic_AI_Travel_Planner/content/drive/MyDrive/Agentic_AI_Travel_Planner/flights.json


In [6]:
import os

data_path = "/content/drive/MyDrive/Agentic_AI_Travel_Planner"
os.listdir(data_path)


['flights.json', 'hotels.json', 'places.json']

In [7]:
# STEP 3C: Drive se JSON Load
import json

def load_json_from_drive(file_name):
    try:
        with open(f"{data_path}/{file_name}", "r") as f:
            return json.load(f)
    except Exception as e:
        print(f" Error loading {file_name}:", e)
        return []

flights_data = load_json_from_drive("flights.json")
hotels_data  = load_json_from_drive("hotels.json")
places_data  = load_json_from_drive("places.json")

print("Flights:", len(flights_data))
print("Hotels :", len(hotels_data))
print("Places :", len(places_data))


Flights: 30
Hotels : 40
Places : 40


In [8]:
# STEP 3D: Data Structure Verify
flights_data[0]


{'flight_id': 'FL0001',
 'airline': 'IndiGo',
 'from': 'Hyderabad',
 'to': 'Delhi',
 'departure_time': '2025-01-04T11:32:00',
 'arrival_time': '2025-01-04T15:32:00',
 'price': 2907}

In [9]:
# hotel sample
hotels_data[0]

{'hotel_id': 'HOT0001',
 'name': 'Grand Palace Hotel',
 'city': 'Delhi',
 'stars': 4,
 'price_per_night': 3897,
 'amenities': ['wifi', 'pool']}

In [10]:
# Place sample
places_data[0]


{'place_id': 'PLC0001',
 'name': 'Famous Fort',
 'city': 'Delhi',
 'type': 'lake',
 'rating': 4.6}

In [11]:
# STEP 4 (UPDATED): Hotel Recommendation Tool (FIXED)
def recommend_hotel(city):
    city_hotels = [
        h for h in hotels_data
        if h["city"].lower() == city.lower()
    ]

    if not city_hotels:
        return None

    # Highest stars first, then lowest price
    best_hotel = sorted(
        city_hotels,
        key=lambda x: (-x["stars"], x["price_per_night"])
    )[0]

    return best_hotel


In [12]:
hotel = recommend_hotel("Delhi")
hotel



{'hotel_id': 'HOT0002',
 'name': 'Comfort Suites',
 'city': 'Delhi',
 'stars': 5,
 'price_per_night': 3650,
 'amenities': ['gym', 'breakfast', 'wifi', 'parking']}

In [13]:
# STEP 5 (UPDATED): Places Recommendation Tool (FIXED)
def recommend_places(city, limit=3):
    city_places = [
        p for p in places_data
        if p["city"].lower() == city.lower()
    ]

    # Highest rating first
    city_places = sorted(
        city_places,
        key=lambda x: x["rating"],
        reverse=True
    )

    return city_places[:limit]


In [14]:
places = recommend_places("Delhi")

for p in places:
    print(p["name"], "-", p["rating"])


Famous Fort - 4.6
Popular Museum - 4.5
Beautiful Temple - 4.2


In [15]:
# STEP 6: Budget Function (HOTEL FIX)
def calculate_budget(flight, hotel, days):
    flight_cost = flight["price"]
    hotel_cost = hotel["price_per_night"] * days
    food_local = 800 * days

    total = flight_cost + hotel_cost + food_local

    return {
        "flight": flight_cost,
        "hotel": hotel_cost,
        "food_travel": food_local,
        "total": total
    }


In [16]:
# STEP 7A: LangChain Imports
from langchain.tools import tool

In [17]:
# STEP 7B: Flight Search Tool (LangChain Tool)
@tool
def flight_search_tool(source: str, destination: str) -> dict:
    """
    Find cheapest flight between two cities (case & space safe).
    """
    source = source.strip().lower()
    destination = destination.strip().lower()

    matches = []

    for f in flights_data:
        from_city = str(f.get("from_city", "")).strip().lower()
        to_city   = str(f.get("to_city", "")).strip().lower()

        if source in from_city and destination in to_city:
            matches.append(f)

    if not matches:
        return {
            "error": "No flights found",
            "available_from_cities": list(set(f.get("from_city") for f in flights_data[:10])),
            "available_to_cities": list(set(f.get("to_city") for f in flights_data[:10]))
        }

    cheapest = min(matches, key=lambda x: x["price"])
    return cheapest


In [18]:
flight_search_tool.run({"source": "Delhi", "destination": "Goa"})


{'error': 'No flights found',
 'available_from_cities': [None],
 'available_to_cities': [None]}

In [19]:
flights_data[0]


{'flight_id': 'FL0001',
 'airline': 'IndiGo',
 'from': 'Hyderabad',
 'to': 'Delhi',
 'departure_time': '2025-01-04T11:32:00',
 'arrival_time': '2025-01-04T15:32:00',
 'price': 2907}

In [20]:
for f in flights_data[:5]:
    print(f)


{'flight_id': 'FL0001', 'airline': 'IndiGo', 'from': 'Hyderabad', 'to': 'Delhi', 'departure_time': '2025-01-04T11:32:00', 'arrival_time': '2025-01-04T15:32:00', 'price': 2907}
{'flight_id': 'FL0002', 'airline': 'Air India', 'from': 'Delhi', 'to': 'Kolkata', 'departure_time': '2025-11-26T05:34:00', 'arrival_time': '2025-11-26T09:34:00', 'price': 3779}
{'flight_id': 'FL0003', 'airline': 'SpiceJet', 'from': 'Chennai', 'to': 'Hyderabad', 'departure_time': '2025-06-03T00:26:00', 'arrival_time': '2025-06-03T01:26:00', 'price': 5473}
{'flight_id': 'FL0004', 'airline': 'Air India', 'from': 'Bangalore', 'to': 'Mumbai', 'departure_time': '2025-05-13T13:18:00', 'arrival_time': '2025-05-13T17:18:00', 'price': 4764}
{'flight_id': 'FL0005', 'airline': 'Air India', 'from': 'Chennai', 'to': 'Bangalore', 'departure_time': '2025-02-08T03:08:00', 'arrival_time': '2025-02-08T05:08:00', 'price': 3695}


In [21]:
# First flight record ka full structure
flights_data[0].keys()


dict_keys(['flight_id', 'airline', 'from', 'to', 'departure_time', 'arrival_time', 'price'])

In [22]:
flights_data[0]


{'flight_id': 'FL0001',
 'airline': 'IndiGo',
 'from': 'Hyderabad',
 'to': 'Delhi',
 'departure_time': '2025-01-04T11:32:00',
 'arrival_time': '2025-01-04T15:32:00',
 'price': 2907}

In [23]:
# STEP 8: FLIGHT TOOL — FINAL & PERMANENT FIX
from langchain.tools import tool

@tool
def flight_search_tool(source: str, destination: str) -> dict:
    """
    Find cheapest flight between two cities using flights.json.
    """
    source = source.strip().lower()
    destination = destination.strip().lower()

    matches = [
        f for f in flights_data
        if str(f.get("from", "")).strip().lower() == source
        and str(f.get("to", "")).strip().lower() == destination
    ]

    if not matches:
        return {
            "error": "No flights found",
            "hint": "Check source/destination spelling in flights.json"
        }

    cheapest = min(matches, key=lambda x: x["price"])
    return cheapest


In [24]:
# STEP 9: TEST (IMPORTANT)
flight_search_tool.run({
    "source": "Hyderabad",
    "destination": "Delhi"
})


{'flight_id': 'FL0001',
 'airline': 'IndiGo',
 'from': 'Hyderabad',
 'to': 'Delhi',
 'departure_time': '2025-01-04T11:32:00',
 'arrival_time': '2025-01-04T15:32:00',
 'price': 2907}

In [25]:
# Hotel Tool ko RE-DEFINE
from langchain.tools import tool

@tool
def hotel_recommendation_tool(city: str) -> dict:
    """
    Recommend best hotel based on stars and price.
    """
    city_hotels = [
        h for h in hotels_data
        if h["city"].lower() == city.lower()
    ]

    if not city_hotels:
        return {"error": "No hotels found"}

    best_hotel = sorted(
        city_hotels,
        key=lambda x: (-x["stars"], x["price_per_night"])
    )[0]

    return best_hotel


In [26]:
@tool
def places_discovery_tool(city: str, days: int = 3) -> list:
    """
    Recommend top places based on rating.
    """
    city_places = [
        p for p in places_data
        if p["city"].lower() == city.lower()
    ]

    city_places = sorted(
        city_places,
        key=lambda x: x["rating"],
        reverse=True
    )

    return city_places[:days]


In [27]:
# Places Tool bhi CONFIRM
places_discovery_tool


StructuredTool(name='places_discovery_tool', description='Recommend top places based on rating.', args_schema=<class 'langchain_core.utils.pydantic.places_discovery_tool'>, func=<function places_discovery_tool at 0x7d01a3bce700>)

In [28]:
@tool
def weather_tool(latitude: float, longitude: float) -> list:
    """
    Get 3-day weather forecast (max temperature).
    """
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&daily=temperature_2m_max&timezone=auto"
    response = requests.get(url)

    if response.status_code != 200:
        return ["Weather data unavailable"]

    data = response.json()
    return data["daily"]["temperature_2m_max"][:3]


In [29]:
# Weather Tool CONFIRM
weather_tool


StructuredTool(name='weather_tool', description='Get 3-day weather forecast (max temperature).', args_schema=<class 'langchain_core.utils.pydantic.weather_tool'>, func=<function weather_tool at 0x7d018418cd60>)

In [30]:
# Tools List
tools = [
    flight_search_tool,
    hotel_recommendation_tool,
    places_discovery_tool,
    weather_tool
]

tools


[StructuredTool(name='flight_search_tool', description='Find cheapest flight between two cities using flights.json.', args_schema=<class 'langchain_core.utils.pydantic.flight_search_tool'>, func=<function flight_search_tool at 0x7d01a2b52020>),
 StructuredTool(name='hotel_recommendation_tool', description='Recommend best hotel based on stars and price.', args_schema=<class 'langchain_core.utils.pydantic.hotel_recommendation_tool'>, func=<function hotel_recommendation_tool at 0x7d01a0daa980>),
 StructuredTool(name='places_discovery_tool', description='Recommend top places based on rating.', args_schema=<class 'langchain_core.utils.pydantic.places_discovery_tool'>, func=<function places_discovery_tool at 0x7d01a3bce700>),
 StructuredTool(name='weather_tool', description='Get 3-day weather forecast (max temperature).', args_schema=<class 'langchain_core.utils.pydantic.weather_tool'>, func=<function weather_tool at 0x7d018418cd60>)]

In [31]:
# STEP 10: ALL TOOLS KO EK LIST ME BIND
tools = [
    flight_search_tool,
    hotel_recommendation_tool,
    places_discovery_tool,
    weather_tool
]

tools


[StructuredTool(name='flight_search_tool', description='Find cheapest flight between two cities using flights.json.', args_schema=<class 'langchain_core.utils.pydantic.flight_search_tool'>, func=<function flight_search_tool at 0x7d01a2b52020>),
 StructuredTool(name='hotel_recommendation_tool', description='Recommend best hotel based on stars and price.', args_schema=<class 'langchain_core.utils.pydantic.hotel_recommendation_tool'>, func=<function hotel_recommendation_tool at 0x7d01a0daa980>),
 StructuredTool(name='places_discovery_tool', description='Recommend top places based on rating.', args_schema=<class 'langchain_core.utils.pydantic.places_discovery_tool'>, func=<function places_discovery_tool at 0x7d01a3bce700>),
 StructuredTool(name='weather_tool', description='Get 3-day weather forecast (max temperature).', args_schema=<class 'langchain_core.utils.pydantic.weather_tool'>, func=<function weather_tool at 0x7d018418cd60>)]

In [32]:
# OpenAI API KEY SET KARO (SECURE WAY)
import os
os.environ["OPENAI_API_KEY"] = "PASTE_YOUR_OPENAI_KEY_HERE"


In [34]:
# City → Latitude Mapping
CITY_COORDS = {
    "delhi": (28.6139, 77.2090),
    "hyderabad": (17.3850, 78.4867),
    "goa": (15.2993, 74.1240)
}


In [35]:
# Agentic Planner Function
def agentic_travel_planner(source, destination, days):
    print(f"\n🧳 Planning {days}-Day Trip: {source} → {destination}\n")

    # 1️⃣ Flight
    flight = flight_search_tool.run({
        "source": source,
        "destination": destination
    })

    if "error" in flight:
        print("❌ Flight Error:", flight["error"])
        return

    # 2️⃣ Hotel
    hotel = hotel_recommendation_tool.run({
        "city": destination
    })

    if "error" in hotel:
        print("❌ Hotel Error:", hotel["error"])
        return

    # 3️⃣ Places
    places = places_discovery_tool.run({
        "city": destination,
        "days": days
    })

    # 4️⃣ Weather
    lat, lon = CITY_COORDS.get(destination.lower(), (None, None))
    weather = weather_tool.run({
        "latitude": lat,
        "longitude": lon
    })

    # 5️⃣ Budget
    total_budget = (
        flight["price"] +
        hotel["price_per_night"] * days +
        800 * days
    )

    # 🔥 FINAL OUTPUT
    print("✈️ Flight Selected:")
    print(f"  {flight['airline']} | ₹{flight['price']}")

    print("\n🏨 Hotel Selected:")
    print(f"  {hotel['name']} | ⭐{hotel['stars']} | ₹{hotel['price_per_night']}/night")

    print("\n📍 Itinerary:")
    for i, p in enumerate(places, 1):
        print(f"  Day {i}: {p['name']}")

    print("\n🌦 Weather (Max Temp):", weather)
    print("\n💰 Estimated Budget: ₹", total_budget)


In [36]:
# RUN THE AGENT (FINAL DEMO)
agentic_travel_planner(
    source="Hyderabad",
    destination="Delhi",
    days=2
)



🧳 Planning 2-Day Trip: Hyderabad → Delhi

✈️ Flight Selected:
  IndiGo | ₹2907

🏨 Hotel Selected:
  Comfort Suites | ⭐5 | ₹3650/night

📍 Itinerary:
  Day 1: Famous Fort
  Day 2: Popular Museum

🌦 Weather (Max Temp): [22.1, 20.9, 22.2]

💰 Estimated Budget: ₹ 11807


In [37]:
# FINAL OUTPUT KO JSON FORMAT
def agentic_travel_planner_json(source, destination, days):
    flight = flight_search_tool.run({"source": source, "destination": destination})
    hotel = hotel_recommendation_tool.run({"city": destination})
    places = places_discovery_tool.run({"city": destination, "days": days})
    lat, lon = CITY_COORDS.get(destination.lower(), (None, None))
    weather = weather_tool.run({"latitude": lat, "longitude": lon})

    budget = {
        "flight": flight["price"],
        "hotel": hotel["price_per_night"] * days,
        "food_travel": 800 * days,
        "total": flight["price"] + hotel["price_per_night"] * days + 800 * days
    }

    return {
        "trip_summary": {
            "source": source,
            "destination": destination,
            "days": days
        },
        "flight": flight,
        "hotel": hotel,
        "itinerary": places,
        "weather": weather,
        "budget": budget
    }


In [38]:
agentic_travel_planner_json("Hyderabad", "Delhi", 2)


{'trip_summary': {'source': 'Hyderabad', 'destination': 'Delhi', 'days': 2},
 'flight': {'flight_id': 'FL0001',
  'airline': 'IndiGo',
  'from': 'Hyderabad',
  'to': 'Delhi',
  'departure_time': '2025-01-04T11:32:00',
  'arrival_time': '2025-01-04T15:32:00',
  'price': 2907},
 'hotel': {'hotel_id': 'HOT0002',
  'name': 'Comfort Suites',
  'city': 'Delhi',
  'stars': 5,
  'price_per_night': 3650,
  'amenities': ['gym', 'breakfast', 'wifi', 'parking']},
 'itinerary': [{'place_id': 'PLC0001',
   'name': 'Famous Fort',
   'city': 'Delhi',
   'type': 'lake',
   'rating': 4.6},
  {'place_id': 'PLC0004',
   'name': 'Popular Museum',
   'city': 'Delhi',
   'type': 'lake',
   'rating': 4.5}],
 'weather': [22.1, 20.9, 22.2],
 'budget': {'flight': 2907,
  'hotel': 7300,
  'food_travel': 1600,
  'total': 11807}}

In [45]:
!pip install streamlit
!pip install pyngrok

In [46]:
import streamlit as st
print("Streamlit Installed")


Streamlit Installed


In [49]:
from pyngrok import ngrok
ngrok.set_auth_token("34pBZArucIzwiqZA2y1JZjVBDBj_2o3vCZUEdbHzeuW5h9nfi")


In [66]:
%%writefile app.py
import streamlit as st

st.set_page_config(page_title="Agentic AI Travel Planner", layout="centered")

st.title("✈️ Agentic AI Travel Planner")
st.caption("Agentic AI-Based Travel Planning System")

# =====================
# BASIC TRIP INPUT
# =====================
source = st.text_input("Source City", "Hyderabad")
destination = st.text_input("Destination City", "Delhi")
days = st.slider("Trip Duration (Days)", 1, 7, 2)

st.divider()

# =====================
# FLIGHT OPTIONS
# =====================
st.subheader("🛫 Flight Preferences")

flight_id = st.text_input("Flight ID", "FL0001")
airline = st.selectbox("Airline", ["IndiGo", "Air India", "Vistara"])
travel_class = st.selectbox("Class", ["Economy", "Business"])
stops = st.selectbox("Stops", [0, 1, 2])
refundable = st.checkbox("Refundable Ticket")
meal = st.checkbox("Meal Included")

st.divider()

# =====================
# HOTEL OPTIONS
# =====================
st.subheader("🏨 Hotel Preferences")

hotel_name = st.selectbox(
    "Hotel Name",
    ["Comfort Suites", "Grand Palace Hotel"]
)
stars = st.selectbox("Hotel Stars", [3, 4, 5])
price_per_night = st.number_input(
    "Price per Night (₹)",
    min_value=1000,
    value=3500
)

amenities = st.multiselect(
    "Amenities",
    ["wifi", "breakfast", "gym", "pool", "parking"],
    default=["wifi", "breakfast"]
)

st.divider()

# =====================
# PLACES OPTIONS
# =====================
st.subheader("📍 Places to Visit")

places = st.multiselect(
    "Select Places",
    ["Famous Fort", "Popular Museum", "City Market", "Heritage Palace"],
    default=["Famous Fort", "Popular Museum"]
)

st.divider()

# =====================
# PLAN TRIP
# =====================
if st.button("Plan Trip"):
    st.success("Trip Planned Successfully ✅")

    # Flight Data
    st.subheader("✈️ Flight Details")
    flight_data = {
        "flight_id": flight_id,
        "airline": airline,
        "from": source,
        "to": destination,
        "class": travel_class,
        "stops": stops,
        "refundable": refundable,
        "meal": meal,
        "price": 2907
    }
    st.json(flight_data)

    # Hotel Data
    st.subheader("🏨 Hotel Details")
    hotel_data = {
        "name": hotel_name,
        "stars": stars,
        "price_per_night": price_per_night,
        "amenities": amenities
    }
    st.json(hotel_data)

    # Itinerary
    st.subheader("📅 Day-wise Itinerary")
    for i, place in enumerate(places, start=1):
        st.write(f"Day {i}: {place}")

    # Budget
    st.subheader("💰 Budget Breakdown")
    hotel_cost = price_per_night * days
    total_budget = 2907 + hotel_cost

    budget = {
        "flight_cost": 2907,
        "hotel_cost": hotel_cost,
        "total_estimated_budget": total_budget
    }
    st.json(budget)


Overwriting app.py


In [67]:
from pyngrok import ngrok
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://crystallizable-audie-compressed.ngrok-free.dev" -> "http://localhost:8501">